Now in dabbage_step1_prep.py
----------------

 - Remove elevation +/- 50m of the average of the urban area
   - Get the average with vectorlayers from the elevation dem file
   - For each vector and corresponding buffer:
     - Clip to the area of the buffer, set any values +/- 50m to -9999
     - ~~Clip against the entire urban area was done in the query~~

In [1]:
import pickle
with open('Dabbage/postsampleplacescall.pickle', 'rb') as fout:
    sampleplaces = pickle.load(fout)

In [13]:
s = None
for i in sampleplaces.values():
    if i[u'usgsplacename'] == "Chicago":
        s = i
        break
print s


{u'usgsplacename': u'Chicago', 'earthenv': {'geom': <shapely.geometry.polygon.Polygon object at 0x10ce0d190>, 'ruralgeomwgs84': <shapely.geometry.multipolygon.MultiPolygon object at 0x10ce0d150>, 'wgs84': <shapely.geometry.polygon.Polygon object at 0x10ce0d110>, 'ruralgeom': <shapely.geometry.multipolygon.MultiPolygon object at 0x10ce0d1d0>}, u'statefips': u'17', u'usgsplacegeomstr': '\x01\x04\x00\x00 \x11\x0f\x00\x00\x01\x00\x00\x00\x01\x01\x00\x00\x00R\xce\x14\xe6<\x9cb\xc1\xed\n3a\x1b\x9aSA', u'placeid': 6648, 'usgsplacegeom': <shapely.geometry.multipoint.MultiPoint object at 0x10ce0d210>, u'countryfips': u'031', u'gnisid': 423587.0, u'usgsplacegeomwgs84str': '\x01\x04\x00\x00 \xe6\x10\x00\x00\x01\x00\x00\x00\x01\x01\x00\x00\x00\x1cL\xf6t\x9a\xe9U\xc07\xbf\x9d\xe1\xcd\xecD@', 'usgsplacegeomwgs84': <shapely.geometry.multipoint.MultiPoint object at 0x10ce0d250>, u'usgspopulation': 2695598.0, 'censusurb': {'geom': <shapely.geometry.multipolygon.MultiPolygon object at 0x10ce0d310>, 'rur

In [55]:
import rasterio
from rasterio.mask import mask

from shapely.geometry import MultiPolygon, shape
from rasterio.features import shapes
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
import numpy as np
import pandas as pd
import json
import os
from pyspatial.raster import read_catalog
from pyspatial.vector import from_series
# import matplotlib.pyplot as plt
from rasterio import plot
from rasterio.transform import from_bounds

demfile = "/Volumes/UrbisBackup/rasterstorage/dem/dem.tif"
demsrc = rasterio.open(demfile)
profile = demsrc.profile

dempyspatialpath = "/Volumes/UrbisBackup/rasterstorage/dem/dem.json"
dempyspatial = read_catalog(dempyspatialpath)




for s in sampleplaces.values():
    print "doing", s[u'usgsplacename']
    if s['censusurb'].get('processbuffer', False):
        continue
    try:

        buffergeom = mapping(s['censusurb']['ruralgeom'])

        try:
            os.mkdir('scratch')
        except:
            pass

        #get average elevation of the urbangeom
        urbanvl = from_series(pd.Series([s['censusurb']['geom']]))
        urbanelevationresult = dempyspatial.query(urbanvl) 
        r = urbanelevationresult.next()
        indexc = np.argwhere(r.values > -9000)
        newv =  np.take(r.values, indexc)
        neww =  np.take(r.weights, indexc)
        urbmeanelevation = float((newv * neww).sum() / neww.sum())

        # now calculate the buffer section
        result = mask(demsrc, [buffergeom], nodata=None, crop=True, all_touched=False, invert=False)
        z = np.where((result[0] > urbmeanelevation - 50) & (result[0] < urbmeanelevation + 50), 1, 0)
        p = z[0].astype('uint8')


        w,south, e, n = list(s['censusurb']['ruralgeom'].bounds)
        bufferaffine = from_bounds(w,south,e,n, z[0].shape[0], z[0].shape[1])



        with rasterio.open('scratch/rgb_no_ndv.tif', 'w', driver=profile['driver'], 
                           width=z[0].shape[0], height=z[0].shape[1], count=1, dtype='uint8', nodata=0,
                          affine=bufferaffine, crs=profile['crs']) as dst:
            dst.write(p, 1)



        with rasterio.open('scratch/rgb_no_ndv.tif', 'r') as newsrc:
            elevationoutput = newsrc.read(1)
            tempgeoms = []
            for shaper in shapes(elevationoutput, mask= elevationoutput==1, transform=bufferaffine):
                tempgeoms.append(shaper)

        multipolygons = MultiPolygon([shape(g[0]) for g in geomobjs])

        s['censusurb']['processbuffer'] = cascaded_union(multipolygons)
    except Exception,e:
        print e
        print "---------"





doing Wichita
doing Springfield
doing Raleigh
doing Dallas
doing Lancaster
doing Grand Rapids
doing Modesto
doing Houston
doing Columbus
doing Rochester
doing Lincoln
'NoneType' object has no attribute 'astype'
---------
doing Anchorage
doing Albuquerque
doing Kansas City
doing Madison
doing Corpus Christi
doing Santa Rosa
doing Denver
doing Fresno
doing Tampa
doing Miami
doing Minneapolis
doing Charlotte
doing Richmond
doing Montgomery
doing El Paso
doing Little Rock
doing Virginia Beach
doing Indianapolis
doing Bakersfield
doing Portland
doing Mobile
doing Greensboro
doing Jackson
doing Sioux Falls
doing Boston
doing Philadelphia
doing Cape Coral
doing Winston-Salem
doing Chattanooga
doing Salt Lake City
doing Cleveland
doing Fort Wayne
doing Milwaukee
doing Baton Rouge
doing Newport News
doing Shreveport
doing San Diego
doing Spokane
doing Las Vegas
doing Reno
doing Lubbock
doing Saint Louis
doing Buffalo
doing Cincinnati
doing Atlanta
doing Nashville
doing Orlando
doing Birmingham


In [56]:
import pickle
with open('Dabbage/postelevationremoval.pickle', 'wb') as fout:
    pickle.dump(sampleplaces, fout)

In [30]:

from pyspatial.vector import from_series

z = from_series(pd.Series([multipolygons]))




In [36]:
from fiona.crs import from_epsg
import fiona
yourschema =  {'geometry': 'Polygon',
           'properties': {}}
with fiona.open('scratch/elevationoutput_shapefile.shp', 'w',
                crs=from_epsg(3857),driver='ESRI Shapefile', 
                schema=yourschema) as output:
    for g in shapeiter:
        output.write({'geometry': g[0], 'properties':{}})
